In [1]:
from extract_parquet import read_parquet,extract_file_path
import cmbagent 
from make_prompts import make_prompt, extract_python_definition
import os
import pandas as pd

In [2]:
%load_ext autoreload
%autoreload 2
from ins import evaluate_camb

In [3]:
# Example with a single file
df = pd.read_csv('data/camb_data_v2.csv')


In [8]:
df

,Unnamed: 0,statement,prompt,reference_code,output
0,0,Compute the CMB Temperature power spectrum for...,###Problem\nCompute the CMB temperature power ...,import camb\nimport numpy as np\ndef get_tt(om...,"(array([ 0, 1, 2, ..., 2998, 2999, 300..."
1,1,Compute the CMB Temperature power spectrum for...,### Problem:\nCompute the CMB temperature powe...,import camb\nimport numpy as np\ndef get_tt(om...,"(array([ 0, 1, 2, ..., 2998, 2999, 300..."
2,2,Compute the CMB Temperature power spectrum for...,### Problem:\nCompute the CMB temperature powe...,import camb\nimport numpy as np\ndef get_tt(H0...,"(array([ 0, 1, 2, ..., 2998, 2999, 300..."
3,3,Compute the Raw CMB Temperature power spectrum...,### Problem:\nCompute the raw CMB temperature ...,import camb\nimport numpy as np\ndef get_tt(H0...,"(array([ 0, 1, 2, ..., 2998, 2999, 300..."
4,4,Compute the CMB E-mode power spectrum for Plan...,### Problem:\nCompute the CMB E-mode power spe...,import camb\nimport numpy as np\ndef get_ee(ta...,"(array([ 0, 1, 2, ..., 2998, 2999, 300..."
5,5,Compute the total CMB B-mode power spectrum fo...,### Problem:\nCompute the CMB B-mode power spe...,import camb\nimport numpy as np\ndef get_bb(r=...,"(array([ 0, 1, 2, ..., 2998, 2999, 300..."
6,6,Compute the total CMB B-mode power spectrum fo...,### Problem:\nCompute the CMB B-mode power spe...,import camb\nimport numpy as np\ndef get_bb(r=...,"(array([ 0, 1, 2, ..., 2998, 2999, 300..."
7,7,Compute the angular diameter distance for Plan...,### Problem:\nCompute the angular diameter dis...,import camb\nimport numpy as np\ndef get_angul...,"(array([0. , 0.04040404, 0.08080808, 0...."
8,8,Compute the linear matter power spectrum with ...,### Problem:\nCompute the linear matter power ...,import camb\nimport numpy as np\ndef get_matte...,"(array([1.00000000e-04, 1.04737090e-04, 1.0969..."
9,9,Compute the delensed total CMB B-mode power sp...,### Problem:\nCompute the Delensed CMB B-mode ...,import camb\nimport numpy as np\ndef get_bb_de...,"(array([ 0, 1, 2, ..., 2998, 2999, 300..."


In [4]:
i = 0
prompt = df.prompt.iloc[i]

In [5]:
reference_code = df.reference_code.iloc[i]

In [6]:
reference_code

"import camb\nimport numpy as np\ndef get_tt(ombh2=0.02):\n    H0=67.5\n    omch2=0.122\n    mnu=0.06\n    omk=0\n    tau=0.06\n    lmax=3000\n    pars = camb.set_params(H0=H0, ombh2=ombh2, omch2=omch2, mnu=mnu, omk=omk, tau=tau,lmax=lmax+500)\n    results = camb.get_results(pars)\n    powers =results.get_cmb_power_spectra(pars, CMB_unit='muK')\n    spectra = powers['total']\n    return np.arange(lmax+1), spectra[:lmax+1,0]\n"

In [12]:
results = cmbagent.one_shot(prompt,
                   max_rounds=50,
                   agent='camb_context',
                   #agent='engineer',
                   #engineer_model='gpt-4.1',
                   #researcher_model='gpt-4.1',
                   #engineer_model='gemini-2.5-pro-exp-03-25',
                            
                   work_dir="./evals",
                  )

_User (to chat_manager):

###Problem
Compute the CMB temperature power spectra in uK^2 units for a baryon density parameter set to 0.02, up to a maximum multipole of 3000. Consider other parameters such as H0=67.3, omch2=0.122, mnu=0.06, omk=0, tau=0.06.

###Requirement
provide a python code where ombh2 can be variable like
\begin{python}
def get_tt(ombh2=0.02):
    return l, cl
\end{python}


--------------------------------------------------------------------------------

Next speaker: camb_context


>>>>>>>> USING AUTO REPLY...
                       Model        agent    Cost  Prompt Tokens  Completion Tokens  Total Tokens
gemini-2.5-pro-preview-03-25 camb_context 0.15478          66896               7116         74012
camb_context (to chat_manager):

**File Search Task:**

The task is to find the necessary CAMB package methods and parameters to compute the CMB temperature power spectra ($C_\ell^{TT}$) in $\mu K^2$ units. The computation needs to support a variable baryon density (

In [15]:
filepath = os.path.join(results['final_context']['work_dir'],extract_file_path(results['final_context']['current_codebase']))

In [16]:
filepath

'./evals/codebase/camb_tt_power_spectrum.py'

In [17]:
with open(filepath, "r", encoding="utf-8") as f:
    candidate_code = f.read()

In [18]:
evaluate_camb(reference_code,candidate_code)

Output()

{'no_input_array_io': 1.0, 'content_similarity': 0.2513721185510428}